In [ ]:
from google.colab import files
uploaded = files.upload()


Saving data.txt to data.txt


In [ ]:
from datasets import load_dataset

dataset = load_dataset("text", data_files={"train": "data.txt"})
print(dataset)

def add_labels(example):
    example["labels"] = example["input_ids"].copy()
    return example

tokenized_datasets = tokenized_datasets.map(add_labels)



DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4
    })
})


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
print(tokenized_datasets)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4
    })
})


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-nature",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=1,
    save_steps=5,
    save_total_limit=2,
    report_to="none"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"]
)

trainer.train()
trainer.save_model("/content/gpt2-nature")
tokenizer.save_pretrained("/content/gpt2-nature")


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
1,9.122500
2,6.027800


('/content/gpt2-nature/tokenizer_config.json',
 '/content/gpt2-nature/special_tokens_map.json',
 '/content/gpt2-nature/vocab.json',
 '/content/gpt2-nature/merges.txt',
 '/content/gpt2-nature/added_tokens.json')

In [ ]:
!ls /content


data.txt  gpt2-nature  sample_data


In [ ]:
from transformers import pipeline


generator = pipeline("text-generation", model="/content/gpt2-nature")


outputs = generator(
    "The beauty of nature is",
    max_new_tokens=60,
    num_return_sequences=3,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2
)


for i, output in enumerate(outputs, 1):
    print(f"\nGenerated #{i}:\n{output['generated_text']}")


Device set to use cpu



Generated #1:
The beauty of nature is the freedom it gives to all who love and worship us," says Robert P. Wicks, curator at Kegelsky Gallery in Seattle's Ballard neighborhood where some 630 species live under trees along a large grassy hilltop overlooking Northlake Creek near Lake Washington (Photo: Susan Acker)

Generated #2:
The beauty of nature is that we do not go in and out. We live it by our own power, through the guidance of Nature herself."


Generated #3:
The beauty of nature is to create these moments in which the forces that are doing us harm, such as humans and animals.
As we look back on our time with this magnificent species today, I cannot help but wonder what it means for a human being like you once was—as an avid environmentalist? A hunter who
